### IMPORT LIBRARY

In [13]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event,text
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

# set to_email
to_email = ['nthieu@savills.com.vn','pthihuongnguyen@savills.com.vn']
# to_email = ['nthieu@savills.com.vn']

### CONNECT TO AZURE SQL

In [14]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

#Test Connection
try:
    conn = engine.connect()
    result = conn.execute(text("SELECT 1"))
    print("CONNECTION SUCESSFUL!")
except Exception as e:
    print("CONNECTION FAILED:",str(e))

CONNECTION SUCESSFUL!


### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Ben Tre Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Vinh Long Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/An Giang Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Lao Cai Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Can Tho City
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Thua Thien Hue Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Khanh Hoa Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Country
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Dong Nai Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Quang Ngai Province
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Ho Chi Minh City
/sites/BIHub/Shared Documents/Advisory

In [10]:
# Lấy danh sách tỉnh
province_list = []
for i in list_folder:
    province_list.append(i.split('/')[7])

# Chọn tỉnh
selected_provinces = []

# Input tỉnh
input_string = input("Nhập tên các tỉnh bạn muốn chọn, cách nhau bằng dấu phẩy (nhập -1 để thoát): ")
input_list = input_string.split(',')

for input_province in input_list:
    input_province = input_province.strip().lower()
    if input_province == "-1":
        break

    matched_provinces = [province for province in province_list if input_province in province.lower()]
    if matched_provinces:
        selected_provinces.extend(matched_provinces)
    else:
        print(f"Không tìm thấy tỉnh chứa từ khóa '{input_province}'.")

# Input Năm và Quý
quarter_year = input("Nhập chuỗi năm và quý (ví dụ: 2023Q1): ")
Year, Quarter = quarter_year[:4], quarter_year[4:]

# In danh sách tỉnh đã chọn, năm và quý
print("Các tỉnh đã chọn:")
for province in selected_provinces:
    print(colored(province,'yellow'))
print(colored("Year:{}, Quarter:{}".format(Year,Quarter),'yellow'))

# Tìm các folder chứa từ khóa
selected_province_folders = [folder for folder in list_folder if any(re.search(keyword, folder, re.IGNORECASE) for keyword in selected_provinces)]
selected_year_quarter = []
for folder in selected_province_folders:
    folder_sub2 = folder + '/' + Year + '/' + Quarter
    selected_year_quarter.append(folder_sub2)
# In danh sách các folder đã tìm thấy
print("Folder các tỉnh cần import:")
for folder in selected_province_folders:
    print(colored(folder,'yellow'))

#Lấy file trong Năm và Quý
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in selected_year_quarter:
    if i.split('/')[7] in selected_provinces and i.split('/')[8] in Year and i.split('/')[9] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()

#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Nhập tên các tỉnh bạn muốn chọn, cách nhau bằng dấu phẩy (nhập -1 để thoát): hai duong
Nhập chuỗi năm và quý (ví dụ: 2023Q1): 2023Q1
Các tỉnh đã chọn:
Hai Duong Province
Year:2023, Quarter:Q1
Folder các tỉnh cần import:
/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Hai Duong Province
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Hai Duong Province/2023/Q1/Macro_HaiDuong_20230331.xlsx


In [11]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[0].upper()
    if sector not in ('x'):
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/Macro economic/Flat file/Hai Duong Province/2023/Q1/Macro_HaiDuong_20230331.xlsx']

### IMPORT DATA

In [12]:
'''Prepare ingredients'''
columns_that_need_unidecode=['City', 'District', 'Indicator', 'Attribute_lv1', 'Attribute_lv2', 
                             'Indicator_Unit', 'Source'
                            ]
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat_macro = pd.DataFrame()
df_flat_macro = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file
    #-------------------------------------------------------
    '''Validation step'''
    #Remove unfortmated values
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    data, df_dict = check_dictionary(df_dict, file_name, data, 'City', 'City', sector, engine, sp_object)
    data, df_dict = check_dictionary(df_dict, file_name, data, 'Indicator', 'Indicator', sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
        df_temp_flat_macro = pd.concat([df_temp_flat_macro, data], axis=0)
        df_flat_macro = tracking_flat_file(df_temp_flat_macro, sector)
        insert_to_fresh(file_url, data, cnt_str)
    else:
        pass


  0%|          | 0/1 [00:00<?, ?it/s]

Validate succesfully
CONNECTION SUCESSFUL IN INSERT TO FRESH FUNCTION!
Insert Macro_HaiDuong_20230331 to Fresh
Insert Macro_HaiDuong_20230331 to Fresh
Insert Macro_HaiDuong_20230331 to Fresh


100%|██████████| 1/1 [00:04<00:00,  4.33s/it]


### SEND EMAIL

In [13]:
list_df_flat = [df_flat_macro]
if len(df_dict) != 0:
    print(colored('Missing values in dictionary','yellow'))
    df_noti_html = convert_df_to_html(type_html = 2, df = df_dict, cnxn = engine)
    run_email(email_type = 2, user_email = to_email, df_noti_html = df_noti_html)
else:
    print('Imported data succesfully')
    df_flat_html, df_query_html = convert_df_to_html(type_html = 3, list_df = list_df_flat, type_sector = 2, cnxn = engine)
    run_email(type_sector = 'MACRO', email_type = 3, user_email = to_email, df_flat_html = df_flat_html, df_query_html = df_query_html)

Imported data succesfully
Sent email successfully


### TRACKING AUDIT

In [14]:
insert_to_tracking(list_df_flat, 'Tracking_Macro', engine)

Inserted audit tracking sucessfully
